<a href="https://colab.research.google.com/github/yaelecker/English-German-Dutch-political-statements-model/blob/main/Train_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Hybrid Political Text Classification Model
Combines advanced architecture, paraphrasing, and user-friendly interface
"""

# Setup - Install required packages
!pip install -qU transformers sentence-transformers
!pip install pytorch-ranger
!pip install tqdm

In [ ]:

class MultiQueryFixedAttention(nn.Module):
    """Advanced attention mechanism using prototype queries"""
    def __init__(self, input_dim, query_vectors, heads=1):
        super().__init__()
        self.heads = heads
        self.input_dim = input_dim
        self.num_queries = query_vectors.shape[0]

        self.register_buffer('query_vectors', query_vectors.unsqueeze(1))
        self.query = nn.Linear(input_dim, input_dim, bias=False)
        self.key = nn.Linear(input_dim, input_dim, bias=False)
        self.value = nn.Linear(input_dim, input_dim, bias=False)

    def forward(self, x):
        batch_size = x.shape[0]

        # Expand inputs for multi-head, multi-query attention
        x_expanded = x.unsqueeze(1).expand(-1, self.heads, -1)
        x_expanded = x_expanded.unsqueeze(2).repeat(1, 1, self.num_queries, 1)

        # Expand query vectors
        query_vectors_expanded = self.query_vectors.expand(self.num_queries, self.heads, self.input_dim)
        query_vectors_expanded = query_vectors_expanded.repeat(batch_size, 1, 1, 1)

        # Apply transformations
        queries = self.query(query_vectors_expanded)
        keys = self.key(x_expanded)
        values = self.value(x_expanded)

        # Scaled dot-product attention
        attention_scores = torch.matmul(queries, keys.transpose(-2, -1)) / (self.input_dim ** 0.5)
        attention = F.softmax(attention_scores, dim=-1)
        attended = torch.matmul(attention, values).mean(dim=1)

        return attended.view(-1, self.input_dim * self.heads)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from sentence_transformers import SentenceTransformer, losses, InputExample
from transformers import T5Tokenizer, T5ForConditionalGeneration
import pandas as pd
import numpy as np
import math
import time
import io
import re
from tqdm.auto import tqdm
from pytorch_ranger import Ranger
import ipywidgets as widgets
from IPython.display import display

In [ ]:
class HybridTextDataset(Dataset):
    """Enhanced dataset with automatic weighting and prototype integration"""
    def __init__(self, dataframe, add_prototypes=True, paraphrase_rounds=0):
        self.dataframe = dataframe.copy()

        # Calculate dataset balancing weights
        if 'source' in self.dataframe.columns:
            group_sizes = self.dataframe.groupby('source')['source'].transform('size')
            total_rows = len(self.dataframe)
            self.dataframe['weight'] = total_rows / group_sizes
            min_weight = self.dataframe['weight'].min()
            self.dataframe['weight'] = self.dataframe['weight'] / min_weight
        else:
            self.dataframe['weight'] = 1.0

        # Add multilingual prototypes if requested
        if add_prototypes:
            max_weight = self.dataframe['weight'].max() if 'weight' in self.dataframe.columns else 1.0
            prototype_weight = 3 * max_weight
            prototypes = self._generate_prototypes(prototype_weight)
            self.dataframe = pd.concat([prototypes, self.dataframe], ignore_index=True)

        # Mark for paraphrasing
        self.paraphrase_rounds = paraphrase_rounds
        if 'is_prototype' not in self.dataframe.columns:
            self.dataframe['is_prototype'] = False
        if 'is_paraphrased' not in self.dataframe.columns:
            self.dataframe['is_paraphrased'] = False

    def _generate_prototypes(self, weight=3.0):
        """Generate multilingual prototype sentences"""
        MANT_SENTENCES = [
            "We aim to invest in maintaining the current state",
            "We willen investeren in het behoud van de huidige staat",
            "Wir wollen in die Erhaltung des derzeitigen Zustands investieren"
        ]
        PROG_SENTENCES = [
            "We aim to invest in making changes and improving the current state",
            "We willen investeren in het aanbrengen van veranderingen en het verbeteren van de huidige staat",
            "Wir wollen in Veränderungen und die Verbesserung des aktuellen Zustands investieren"
        ]
        PROT_SENTENCES = [
            "We aim to invest in preventing negative changes to the current state",
            "We willen investeren in het voorkomen van negatieve veranderingen in de huidige staat",
            "Wir wollen in die Verhinderung negativer Veränderungen des derzeitigen Zustands investieren"
        ]

        prototype_data = []
        for i in range(3):  # For each language
            prototype_data.extend([
                {
                    'item': PROG_SENTENCES[i], 'prog_human': 1.0, 'mant_human': 0.0, 'prot_human': 0.0,
                    'is_prototype': True, 'weight': weight, 'source': f'prototype_lang_{i}'
                },
                {
                    'item': MANT_SENTENCES[i], 'prog_human': 0.0, 'mant_human': 1.0, 'prot_human': 0.0,
                    'is_prototype': True, 'weight': weight, 'source': f'prototype_lang_{i}'
                },
                {
                    'item': PROT_SENTENCES[i], 'prog_human': 0.0, 'mant_human': 0.0, 'prot_human': 1.0,
                    'is_prototype': True, 'weight': weight, 'source': f'prototype_lang_{i}'
                }
            ])

        return pd.DataFrame(prototype_data)

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        row = self.dataframe.iloc[index]
        sentence = row['item']
        labels = torch.tensor([row['prog_human'], row['mant_human'], row['prot_human']], dtype=torch.float)
        weights = torch.tensor([row['weight']], dtype=torch.float).expand(3)
        return sentence, labels, weights


In [ ]:
class AdvancedParaphraser:
    """T5-based paraphraser with multilingual support"""
    def __init__(self, model_name='google/flan-t5-large', device='cuda'):
        self.device = device
        self.tokenizer = T5Tokenizer.from_pretrained(model_name)
        self.model = T5ForConditionalGeneration.from_pretrained(model_name)
        self.model.to(device)

    def paraphrase_batch(self, texts, prompt="Paraphrase this in its original language: "):
        """Paraphrase a batch of texts"""
        inputs = [f"{prompt}{text}" for text in texts]
        tokenized = self.tokenizer(inputs, max_length=512, truncation=True,
                                 padding='longest', return_tensors="pt")
        tokenized = {k: v.to(self.device) for k, v in tokenized.items()}

        with torch.no_grad():
            outputs = self.model.generate(
                tokenized['input_ids'],
                attention_mask=tokenized['attention_mask'],
                max_length=512,
                num_beams=5,
                early_stopping=True,
                do_sample=True,
                temperature=0.8
            )

        return [self.tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

    def augment_dataset(self, dataset, rounds=2, batch_size=8):
        """Apply multiple rounds of paraphrasing to augment dataset"""
        if rounds == 0:
            return dataset

        print(f"Applying {rounds} rounds of paraphrasing...")
        original_df = dataset.dataframe.copy()

        # FIX: Properly handle is_prototype column
        if 'is_prototype' not in original_df.columns:
            original_df['is_prototype'] = False

        # Convert NaN values to False
        original_df['is_prototype'] = original_df['is_prototype'].fillna(False).astype(bool)

        augmented_dfs = [original_df]

        # Only paraphrase non-prototypes
        paraphrase_mask = ~original_df['is_prototype']  # Now this will work
        paraphrase_df = original_df[paraphrase_mask].copy()

        if len(paraphrase_df) == 0:
            print("⚠️ No non-prototype texts found to paraphrase")
            return dataset

        for round_num in range(rounds):
            print(f"Paraphrasing round {round_num + 1}/{rounds}")

            paraphrased_items = []
            for i in tqdm(range(0, len(paraphrase_df), batch_size)):
                batch = paraphrase_df.iloc[i:i+batch_size]['item'].tolist()
                try:
                    paraphrased_batch = self.paraphrase_batch(batch)
                    paraphrased_items.extend(paraphrased_batch)
                except Exception as e:
                    print(f"⚠️ Error in batch {i//batch_size + 1}: {e}")
                    # Add original texts if paraphrasing fails
                    paraphrased_items.extend(batch)

            # Create new dataframe with paraphrased items
            round_df = paraphrase_df.copy()
            round_df['item'] = paraphrased_items[:len(round_df)]  # Ensure same length
            round_df['is_paraphrased'] = True
            round_df['paraphrase_round'] = round_num + 1

            # Filter out empty paraphrases
            round_df = round_df[round_df['item'].str.strip().astype(bool)]
            augmented_dfs.append(round_df)

        # Combine all dataframes
        final_df = pd.concat(augmented_dfs, ignore_index=True)
        dataset.dataframe = final_df
        print(f"Dataset augmented from {len(original_df)} to {len(final_df)} samples")
        return dataset

In [ ]:
class MultiQueryFixedAttention(nn.Module):
    """Advanced attention mechanism using prototype queries"""
    def __init__(self, input_dim, query_vectors, heads=1):
        super().__init__()
        self.heads = heads
        self.input_dim = input_dim
        self.num_queries = query_vectors.shape[0]

        self.register_buffer('query_vectors', query_vectors.unsqueeze(1))
        self.query = nn.Linear(input_dim, input_dim, bias=False)
        self.key = nn.Linear(input_dim, input_dim, bias=False)
        self.value = nn.Linear(input_dim, input_dim, bias=False)

    def forward(self, x):
        batch_size = x.shape[0]

        # Expand inputs for multi-head, multi-query attention
        x_expanded = x.unsqueeze(1).expand(-1, self.heads, -1)
        x_expanded = x_expanded.unsqueeze(2).repeat(1, 1, self.num_queries, 1)

        # Expand query vectors
        query_vectors_expanded = self.query_vectors.expand(self.num_queries, self.heads, self.input_dim)
        query_vectors_expanded = query_vectors_expanded.repeat(batch_size, 1, 1, 1)

        # Apply transformations
        queries = self.query(query_vectors_expanded)
        keys = self.key(x_expanded)
        values = self.value(x_expanded)

        # Scaled dot-product attention
        attention_scores = torch.matmul(queries, keys.transpose(-2, -1)) / (self.input_dim ** 0.5)
        attention = F.softmax(attention_scores, dim=-1)
        attended = torch.matmul(attention, values).mean(dim=1)

        return attended.view(-1, self.input_dim * self.heads)

In [ ]:
class EmbeddingWrapper(nn.Module):
    def __init__(self, embedder, trainable):
        super().__init__()
        self.embedder = embedder
        if trainable:
            for param in self.embedder.parameters():
                param.requires_grad = True

    def forward(self, texts):
        return self.embedder.encode(texts, convert_to_tensor=True)

    def get_sentence_embedding_dimension(self):
        return self.embedder.get_sentence_embedding_dimension()

class ParallelEmbedding(nn.Module):
    """Parallel processing of multiple embedding models"""
    def __init__(self, model_names, trainable=True):
        super().__init__()
        self.embedders = nn.ModuleList([
            self._wrap_embedder(SentenceTransformer(name), trainable)
            for name in model_names
        ])

    def _wrap_embedder(self, embedder, trainable):
        """Wrap sentence transformer for gradient computation"""


        return EmbeddingWrapper(embedder, trainable)

    def forward(self, texts):
        embeddings = [embedder(texts) for embedder in self.embedders]
        return torch.cat(embeddings, dim=1) if len(embeddings) > 1 else embeddings[0]

    def get_sentence_embedding_dimension(self):
        return sum(e.get_sentence_embedding_dimension() for e in self.embedders)

In [ ]:
class HybridGoalTypeModel(nn.Module):
    """Hybrid model combining advanced features with practical design"""
    def __init__(self,
                 embedding_models=['paraphrase-multilingual-mpnet-base-v2'],
                 hidden_layers=[512, 256, 128],
                 dropout_rate=0.2,
                 activation=nn.ReLU,
                 use_attention=True,
                 attention_heads=1,
                 batch_norm=True,
                 trainable_embeddings=True):
        super().__init__()

        # Multi-embedding layer
        self.embeddings = ParallelEmbedding(embedding_models, trainable_embeddings)
        embedding_dim = self.embeddings.get_sentence_embedding_dimension()

        layers = []

        # Optional attention mechanism
        if use_attention:
            # Create prototype embeddings for attention
            prototype_texts = [
                "We aim to invest in maintaining the current state",
                "We willen investeren in het behoud van de huidige staat",
                "Wir wollen in die Erhaltung des derzeitigen Zustands investieren",
                "We aim to invest in making changes and improving the current state",
                "We willen investeren in het aanbrengen van veranderingen en het verbeteren van de huidige staat",
                "Wir wollen in Veränderungen und die Verbesserung des aktuellen Zustands investieren",
                "We aim to invest in preventing negative changes to the current state",
                "We willen investeren in het voorkomen van negatieve veranderingen in de huidige staat",
                "Wir wollen in die Verhinderung negativer Veränderungen des derzeitigen Zustands investieren"
            ]

            with torch.no_grad():
                prototype_embeddings = self.embeddings(prototype_texts)

            self.attention = MultiQueryFixedAttention(
                embedding_dim, prototype_embeddings, attention_heads
            )
            layers.append(nn.Dropout(dropout_rate))
            embedding_dim = embedding_dim * attention_heads
        else:
            self.attention = None

        # Hidden layers
        if hidden_layers:
            prev_dim = embedding_dim
            for hidden_dim in hidden_layers:
                layer_components = [nn.Dropout(dropout_rate), nn.Linear(prev_dim, hidden_dim)]
                if batch_norm:
                    layer_components.append(nn.BatchNorm1d(hidden_dim))
                layer_components.append(activation())
                layers.append(nn.Sequential(*layer_components))
                prev_dim = hidden_dim

            final_dim = hidden_layers[-1]
        else:
            final_dim = embedding_dim

        # Output layer
        layers.extend([
            nn.Dropout(dropout_rate),
            nn.Linear(final_dim, 3)
        ])

        self.classifier = nn.Sequential(*layers)

    def forward(self, texts):
        embeddings = self.embeddings(texts)

        if self.attention is not None:
            embeddings = self.attention(embeddings)

        outputs = self.classifier(embeddings)
        return torch.sigmoid(outputs)

In [ ]:
class HybridModelTrainer:
    """Comprehensive trainer with advanced features and user-friendly interface"""
    def __init__(self,
                 embedding_models=['paraphrase-multilingual-mpnet-base-v2'],
                 hidden_layers=[512, 256, 128],
                 use_attention=True,
                 paraphrase_rounds=2,
                 epochs=50,
                 batch_size=32,
                 learning_rate=0.001,
                 device='cuda'):

        self.config = {
            'embedding_models': embedding_models,
            'hidden_layers': hidden_layers,
            'use_attention': use_attention,
            'paraphrase_rounds': paraphrase_rounds,
            'epochs': epochs,
            'batch_size': batch_size,
            'learning_rate': learning_rate
        }

        self.device = torch.device(device if torch.cuda.is_available() else 'cpu')
        self.model = None
        self.optimizer = None
        self.scheduler = None
        self.paraphraser = None

        print(f"Initialized trainer with device: {self.device}")
        print(f"Configuration: {self.config}")

    def setup_ui(self):
        """Create user-friendly interface widgets"""
        # File uploader
        self.data_uploader = widgets.FileUpload(
            accept='.csv',
            multiple=True,
            description='Upload CSV files for training'
        )

        # Training parameters
        self.epochs_slider = widgets.IntSlider(
            value=self.config['epochs'],
            min=10, max=200, step=10,
            description='Training Epochs:'
        )

        self.batch_size_slider = widgets.IntSlider(
            value=self.config['batch_size'],
            min=8, max=128, step=8,
            description='Batch Size:'
        )

        self.paraphrase_slider = widgets.IntSlider(
            value=self.config['paraphrase_rounds'],
            min=0, max=5, step=1,
            description='Paraphrase Rounds:'
        )

        self.attention_checkbox = widgets.Checkbox(
            value=self.config['use_attention'],
            description='Use Attention Mechanism'
        )

        # Model suffix for saving
        self.model_suffix = widgets.Text(
            value='hybrid_model',
            description='Model Name:'
        )

        # Display widgets
        display(widgets.VBox([
            widgets.HTML("<h3>Hybrid Political Text Classification Model</h3>"),
            self.data_uploader,
            self.epochs_slider,
            self.batch_size_slider,
            self.paraphrase_slider,
            self.attention_checkbox,
            self.model_suffix
        ]))

        return self

    def load_data_from_ui(self, train_fraction=0.8):
        """Load data from uploaded files"""
        if not self.data_uploader.value:
            raise ValueError("Please upload CSV files first")

        datasets = []
        for filename, file_info in self.data_uploader.value.items():
            df = pd.read_csv(io.BytesIO(file_info['content']),
                           dtype={'item': str})

            # Normalize columns
            required_cols = ['item', 'prog_human', 'mant_human', 'prot_human']
            if not all(col in df.columns for col in required_cols):
                print(f"Warning: {filename} missing required columns")
                continue

            df = df[required_cols].copy()
            df['item'] = df['item'].astype(str)
            df[['prog_human', 'mant_human', 'prot_human']] = df[['prog_human', 'mant_human', 'prot_human']] / 7.0
            df['source'] = filename

            # Split train/test
            train = df.sample(frac=train_fraction, random_state=42)
            test = df.drop(train.index)
            datasets.append((train, test))

        # Combine datasets
        train_df = pd.concat([d[0] for d in datasets], ignore_index=True)
        test_df = pd.concat([d[1] for d in datasets], ignore_index=True)

        # Create datasets with prototypes
        self.train_dataset = HybridTextDataset(train_df, add_prototypes=True)
        self.test_dataset = HybridTextDataset(test_df, add_prototypes=False)

        print(f"Loaded {len(self.train_dataset)} training and {len(self.test_dataset)} test samples")
        return self

    def apply_data_augmentation(self):
        """Apply paraphrasing for data augmentation"""
        rounds = self.paraphrase_slider.value if hasattr(self, 'paraphrase_slider') else self.config['paraphrase_rounds']

        if rounds > 0:
            if self.paraphraser is None:
                self.paraphraser = AdvancedParaphraser(device=self.device)

            self.train_dataset = self.paraphraser.augment_dataset(
                self.train_dataset, rounds=rounds
            )

        return self

    def build_model(self):
        """Build the hybrid model"""
        # Update config from UI if available
        if hasattr(self, 'attention_checkbox'):
            self.config['use_attention'] = self.attention_checkbox.value
        if hasattr(self, 'batch_size_slider'):
            self.config['batch_size'] = self.batch_size_slider.value
        if hasattr(self, 'epochs_slider'):
            self.config['epochs'] = self.epochs_slider.value

        self.model = HybridGoalTypeModel(
            embedding_models=self.config['embedding_models'],
            hidden_layers=self.config['hidden_layers'],
            use_attention=self.config['use_attention'],
            trainable_embeddings=True,
            batch_norm=True
        ).to(self.device)

        print(f"Built model with {sum(p.numel() for p in self.model.parameters())} parameters")
        return self

    def train_model(self):
        """Train the model with advanced features"""
        if self.model is None:
            self.build_model()

        # Setup training components
        train_loader = DataLoader(
            self.train_dataset,
            batch_size=self.config['batch_size'],
            shuffle=True
        )
        test_loader = DataLoader(
            self.test_dataset,
            batch_size=self.config['batch_size'],
            shuffle=False
        )

        self.optimizer = Ranger(self.model.parameters(), lr=self.config['learning_rate'])
        self.scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
            self.optimizer, T_max=self.config['epochs']//3
        )

        # Training loop with advanced features
        best_loss = float('inf')
        best_epoch = 0

        for epoch in range(self.config['epochs']):
            # Training phase
            self.model.train()
            train_loss = 0

            for sentences, labels, weights in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
                labels = labels.to(self.device)
                weights = weights.to(self.device)

                self.optimizer.zero_grad()
                outputs = self.model(sentences)

                # Weighted loss with NaN handling
                loss = self._weighted_loss(outputs, labels, weights)
                loss.backward()

                # Gradient clipping for stability
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)

                self.optimizer.step()
                self.scheduler.step()
                train_loss += loss.item()

            # Evaluation phase
            test_loss, correlations = self._evaluate(test_loader)

            print(f"Epoch {epoch+1}/{self.config['epochs']}")
            print(f"Train Loss: {train_loss/len(train_loader):.4f}")
            print(f"Test Loss: {test_loss:.4f}")
            print(f"Correlations - Prog: {correlations['prog']:.3f}, "
                  f"Mant: {correlations['mant']:.3f}, Prot: {correlations['prot']:.3f}")

            # Save best model
            if test_loss < best_loss:
                best_loss = test_loss
                best_epoch = epoch + 1
                self._save_checkpoint(epoch, test_loss)
                print(f"*** New best model saved! ***")

        print(f"Training completed. Best model from epoch {best_epoch} with loss {best_loss:.4f}")
        return self

    def _weighted_loss(self, outputs, targets, weights):
        """Weighted MSE loss with NaN handling"""
        mask = ~torch.isnan(targets)
        if not mask.any():
            return torch.tensor(0.0, device=outputs.device)

        outputs_masked = outputs[mask]
        targets_masked = targets[mask]
        weights_masked = weights[mask]

        squared_errors = (outputs_masked - targets_masked) ** 2
        weighted_errors = squared_errors * weights_masked

        return weighted_errors.sum() / weights_masked.sum()

    def _evaluate(self, data_loader):
        """Comprehensive model evaluation"""
        self.model.eval()
        total_loss = 0
        predictions = []
        targets = []

        with torch.no_grad():
            for sentences, labels, weights in data_loader:
                labels = labels.to(self.device)
                weights = weights.to(self.device)
                outputs = self.model(sentences)

                loss = self._weighted_loss(outputs, labels, weights)
                total_loss += loss.item()

                predictions.extend(outputs.cpu().numpy())
                targets.extend(labels.cpu().numpy())

        # Calculate correlations
        predictions = np.array(predictions)
        targets = np.array(targets)

        correlations = {
            'prog': np.corrcoef(predictions[:, 0], targets[:, 0])[0, 1],
            'mant': np.corrcoef(predictions[:, 1], targets[:, 1])[0, 1],
            'prot': np.corrcoef(predictions[:, 2], targets[:, 2])[0, 1]
        }

        # Handle NaN correlations
        for key in correlations:
            if np.isnan(correlations[key]):
                correlations[key] = 0.0

        return total_loss / len(data_loader), correlations

    def _save_checkpoint(self, epoch, loss):
        """Save model checkpoint"""
        suffix = self.model_suffix.value if hasattr(self, 'model_suffix') else 'hybrid_model'

        checkpoint = {
            'epoch': epoch + 1,
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'scheduler_state_dict': self.scheduler.state_dict(),
            'loss': loss,
            'config': self.config
        }

        torch.save(checkpoint, f'{suffix}_best.pth')
        torch.save(self.model, f'{suffix}_complete.pt')

    def predict(self, texts):
        """Make predictions on new texts"""
        if self.model is None:
            raise ValueError("Model not trained yet")

        self.model.eval()
        with torch.no_grad():
            if isinstance(texts, str):
                texts = [texts]
            outputs = self.model(texts)
            return outputs.cpu().numpy()

In [ ]:
def create_flexible_hybrid_interface():
    """Flexible interface with advanced model configuration options"""
    import ipywidgets as widgets
    from IPython.display import display

    trainer = HybridModelTrainer(
        embedding_models=['paraphrase-multilingual-mpnet-base-v2'],
        hidden_layers=[512, 256, 128],
        use_attention=True,
        paraphrase_rounds=0,
        epochs=50,
        batch_size=32
    )

    # File uploader
    data_uploader = widgets.FileUpload(
        accept='.csv',
        multiple=True,
        description='Upload CSV files:'
    )

    # File display area
    file_display = widgets.HTML(value="<b>No files uploaded yet</b>")

    # ==================== MODEL ARCHITECTURE CONTROLS ====================

    # Embedding models selection
    embedding_options = widgets.SelectMultiple(
        options=[
            'paraphrase-multilingual-mpnet-base-v2',
            'BAAI/bge-m3',
            'intfloat/multilingual-e5-large',
            'sentence-transformers/all-MiniLM-L6-v2',
            'sentence-transformers/all-mpnet-base-v2'
        ],
        value=['paraphrase-multilingual-mpnet-base-v2'],
        description='Embedding Models:',
        rows=3
    )

    # Model complexity presets
    complexity_preset = widgets.Dropdown(
        options=[
            ('Simple (No hidden layers)', 'simple'),
            ('Basic (1 layer)', 'basic'),
            ('Medium (2 layers)', 'medium'),
            ('Complex (3 layers)', 'complex'),
            ('Very Complex (4 layers)', 'very_complex'),
            ('Custom', 'custom')
        ],
        value='medium',
        description='Complexity:'
    )

    # Custom hidden layers input
    custom_layers_text = widgets.Text(
        value='512,256,128',
        placeholder='e.g. 512,256,128 or 1024,512',
        description='Custom Layers:',
        disabled=True
    )

    # Attention options
    attention_checkbox = widgets.Checkbox(value=True, description='Use Attention Mechanism')
    attention_heads = widgets.IntSlider(value=1, min=1, max=8, description='Attention Heads:')

    # Advanced options
    trainable_embeddings = widgets.Checkbox(value=True, description='Fine-tune Embeddings')
    batch_norm = widgets.Checkbox(value=True, description='Batch Normalization')
    dropout_rate = widgets.FloatSlider(value=0.2, min=0.0, max=0.8, step=0.1, description='Dropout Rate:')

    # ==================== TRAINING CONTROLS ====================

    epochs_slider = widgets.IntSlider(value=50, min=10, max=300, step=10, description='Epochs:')
    batch_size_slider = widgets.IntSlider(value=32, min=8, max=128, step=8, description='Batch Size:')
    learning_rate = widgets.FloatLogSlider(value=0.001, base=10, min=-5, max=-1, description='Learning Rate:')

    # Data augmentation
    paraphrase_slider = widgets.IntSlider(value=0, min=0, max=5, step=1, description='Paraphrase Rounds:')

    # Model naming
    model_suffix = widgets.Text(value='flexible_model', description='Model Name:')

    # ==================== DYNAMIC UPDATES ====================

    def update_custom_layers_visibility(change):
        """Enable/disable custom layers input based on preset selection"""
        if complexity_preset.value == 'custom':
            custom_layers_text.disabled = False
            custom_layers_text.layout.visibility = 'visible'
        else:
            custom_layers_text.disabled = True
            # Update with preset values
            preset_layers = {
                'simple': '',
                'basic': '256',
                'medium': '512,256',
                'complex': '1024,512,256',
                'very_complex': '1024,512,256,128'
            }
            custom_layers_text.value = preset_layers.get(complexity_preset.value, '512,256,128')

    complexity_preset.observe(update_custom_layers_visibility, names='value')

    def update_attention_heads_visibility(change):
        """Show/hide attention heads slider based on attention checkbox"""
        attention_heads.layout.visibility = 'visible' if attention_checkbox.value else 'hidden'

    attention_checkbox.observe(update_attention_heads_visibility, names='value')

    # ==================== FILE DISPLAY UPDATE ====================

    def update_file_display(change):
        if data_uploader.value:
            file_list = []
            total_size = 0
            for filename, file_info in data_uploader.value.items():
                size_kb = len(file_info['content']) / 1024
                file_list.append(f"📄 {filename} ({size_kb:.1f} KB)")
                total_size += size_kb

            file_display.value = f"""
            <b>Uploaded Files ({len(file_list)}):</b><br>
            {'<br>'.join(file_list)}<br>
            <b>Total size: {total_size:.1f} KB</b>
            """
        else:
            file_display.value = "<b>No files uploaded yet</b>"

    data_uploader.observe(update_file_display, names='value')

    # ==================== CONFIGURATION PREVIEW ====================

    config_preview = widgets.HTML(value="<b>Configuration will appear here</b>")

    def update_config_preview(*args):
        """Update configuration preview when settings change"""

        # Parse hidden layers
        if complexity_preset.value == 'simple':
            hidden_layers = []
        else:
            try:
                if custom_layers_text.value.strip():
                    hidden_layers = [int(x.strip()) for x in custom_layers_text.value.split(',') if x.strip()]
                else:
                    hidden_layers = []
            except:
                hidden_layers = [512, 256, 128]  # fallback

        # Calculate approximate model size
        embedding_dims = {
            'paraphrase-multilingual-mpnet-base-v2': 768,
            'BAAI/bge-m3': 1024,
            'intfloat/multilingual-e5-large': 1024,
            'sentence-transformers/all-MiniLM-L6-v2': 384,
            'sentence-transformers/all-mpnet-base-v2': 768
        }

        total_embedding_dim = sum(embedding_dims.get(model, 768) for model in embedding_options.value)

        # Estimate parameters
        params = total_embedding_dim * 1000000  # Base embedding params (rough estimate)
        current_dim = total_embedding_dim

        if attention_checkbox.value:
            params += current_dim * current_dim * 3 * attention_heads.value  # Q, K, V matrices

        for layer_size in hidden_layers:
            params += current_dim * layer_size + layer_size  # weights + bias
            current_dim = layer_size

        params += current_dim * 3 + 3  # final layer

        config_html = f"""
        <div style="background-color: #f0f0f0; padding: 10px; border-radius: 5px; margin: 5px 0;">
        <b>📋 Current Configuration:</b><br>
        <b>🧠 Architecture:</b><br>
        &nbsp;&nbsp;• Embeddings: {len(embedding_options.value)} models ({total_embedding_dim}D total)<br>
        &nbsp;&nbsp;• Hidden layers: {hidden_layers if hidden_layers else "None (direct classification)"}<br>
        &nbsp;&nbsp;• Attention: {"Yes" if attention_checkbox.value else "No"} {f"({attention_heads.value} heads)" if attention_checkbox.value else ""}<br>
        &nbsp;&nbsp;• Estimated params: ~{params/1000000:.1f}M<br>
        <b>🔧 Training:</b><br>
        &nbsp;&nbsp;• Epochs: {epochs_slider.value}, Batch size: {batch_size_slider.value}<br>
        &nbsp;&nbsp;• Learning rate: {learning_rate.value:.2e}<br>
        &nbsp;&nbsp;• Dropout: {dropout_rate.value:.1f}, Batch norm: {"Yes" if batch_norm.value else "No"}<br>
        &nbsp;&nbsp;• Paraphrasing: {paraphrase_slider.value} rounds<br>
        <b>🎯 Complexity:</b> <span style="color: {'red' if params > 10000000 else 'orange' if params > 5000000 else 'green'}">
        {"Very High" if params > 10000000 else "High" if params > 5000000 else "Moderate" if params > 1000000 else "Low"}</span>
        </div>
        """
        config_preview.value = config_html

    # Observe all relevant widgets for config preview
    for widget in [embedding_options, complexity_preset, custom_layers_text, attention_checkbox,
                   attention_heads, epochs_slider, batch_size_slider, learning_rate,
                   dropout_rate, paraphrase_slider, trainable_embeddings, batch_norm]:
        widget.observe(update_config_preview, names='value')

    # Initial config preview
    update_config_preview()

    # ==================== TRAINING FUNCTION ====================

    train_button = widgets.Button(description='🚀 Start Training', button_style='success', icon='play')
    output_area = widgets.Output()

    def start_flexible_training(b):
        with output_area:
            try:
                # Parse configuration
                if complexity_preset.value == 'simple':
                    hidden_layers = []
                else:
                    try:
                        if custom_layers_text.value.strip():
                            hidden_layers = [int(x.strip()) for x in custom_layers_text.value.split(',') if x.strip()]
                        else:
                            hidden_layers = []
                    except Exception as e:
                        print(f"⚠️ Error parsing hidden layers, using default: {e}")
                        hidden_layers = [512, 256, 128]

                # Create new trainer with flexible configuration
                flexible_trainer = HybridModelTrainer(
                    embedding_models=list(embedding_options.value),
                    hidden_layers=hidden_layers,
                    use_attention=attention_checkbox.value,
                    paraphrase_rounds=paraphrase_slider.value,
                    epochs=epochs_slider.value,
                    batch_size=batch_size_slider.value,
                    learning_rate=learning_rate.value
                )

                # Additional model configuration
                flexible_trainer.config.update({
                    'dropout_rate': dropout_rate.value,
                    'trainable_embeddings': trainable_embeddings.value,
                    'batch_norm': batch_norm.value,
                    'attention_heads': attention_heads.value if attention_checkbox.value else 0
                })

                # Set data uploader
                flexible_trainer.data_uploader = data_uploader
                flexible_trainer.model_suffix = model_suffix

                print("🚀 Starting flexible training with custom configuration...")
                print(f"📊 Architecture: {len(embedding_options.value)} embeddings → {hidden_layers} → 3 outputs")
                print(f"🔧 Attention: {attention_checkbox.value} ({attention_heads.value} heads)")
                print(f"📈 Training: {epochs_slider.value} epochs, batch {batch_size_slider.value}, LR {learning_rate.value:.2e}")

                # Training pipeline
                flexible_trainer.load_data_from_ui()
                if paraphrase_slider.value > 0:
                    flexible_trainer.apply_data_augmentation()
                flexible_trainer.build_model()
                flexible_trainer.train_model()

                print("✅ Flexible training completed successfully!")

                # Update global trainer for analysis
                global trainer
                trainer = flexible_trainer

            except Exception as e:
                print(f"❌ Error during flexible training: {e}")
                import traceback
                traceback.print_exc()

    train_button.on_click(start_flexible_training)

    # ==================== LAYOUT ====================

    ui = widgets.VBox([
        widgets.HTML("<h2>🎛️ Flexible Hybrid Political Text Classification Model</h2>"),

        # File upload section
        widgets.HTML("<h3>📁 Data Upload</h3>"),
        data_uploader,
        file_display,

        # Model architecture section
        widgets.HTML("<h3>🏗️ Model Architecture</h3>"),
        widgets.HBox([
            widgets.VBox([
                widgets.HTML("<b>Embedding Models:</b>"),
                embedding_options,
            ], layout=widgets.Layout(width='50%')),
            widgets.VBox([
                widgets.HTML("<b>Network Structure:</b>"),
                complexity_preset,
                custom_layers_text,
                attention_checkbox,
                attention_heads,
            ], layout=widgets.Layout(width='50%'))
        ]),

        # Advanced options
        widgets.HTML("<h3>⚙️ Advanced Options</h3>"),
        widgets.HBox([
            widgets.VBox([trainable_embeddings, batch_norm]),
            widgets.VBox([dropout_rate])
        ]),

        # Training parameters
        widgets.HTML("<h3>🏋️ Training Parameters</h3>"),
        widgets.HBox([
            widgets.VBox([epochs_slider, batch_size_slider]),
            widgets.VBox([learning_rate, paraphrase_slider])
        ]),

        # Configuration preview
        widgets.HTML("<h3>📋 Configuration Preview</h3>"),
        config_preview,

        # Model naming and training
        widgets.HTML("<h3>🚀 Training</h3>"),
        model_suffix,
        train_button,
        output_area
    ])

    display(ui)
    return trainer



In [ ]:

def create_hybrid_model_interface():
   """Enhanced interface that shows uploaded files"""
   trainer = HybridModelTrainer(
       embedding_models=['paraphrase-multilingual-mpnet-base-v2'],
       hidden_layers=[512, 256, 128],
       use_attention=True,
       paraphrase_rounds=2,
       epochs=50,
       batch_size=32
   )

   # File uploader
   data_uploader = widgets.FileUpload(
       accept='.csv',
       multiple=True,
       description='Upload CSV files:'
   )

   # File display area
   file_display = widgets.HTML(value="<b>No files uploaded yet</b>")

   # Parameter controls
   epochs_slider = widgets.IntSlider(value=50, min=10, max=200, step=10, description='Epochs:')
   batch_size_slider = widgets.IntSlider(value=32, min=8, max=128, step=8, description='Batch Size:')
   paraphrase_slider = widgets.IntSlider(value=2, min=0, max=5, step=1, description='Paraphrase Rounds:')
   attention_checkbox = widgets.Checkbox(value=True, description='Use Attention')
   model_suffix = widgets.Text(value='hybrid_model', description='Model Name:')

   # Train button
   train_button = widgets.Button(description='Start Training', button_style='success', icon='play')

   # Output area
   output_area = widgets.Output()

   # Update file display when files are uploaded
   def update_file_display(change):
       if data_uploader.value:
           file_list = []
           total_size = 0
           for filename, file_info in data_uploader.value.items():
               size_kb = len(file_info['content']) / 1024
               file_list.append(f"📄 {filename} ({size_kb:.1f} KB)")
               total_size += size_kb

           file_display.value = f"""
           <b>Uploaded Files ({len(file_list)}):</b><br>
           {'<br>'.join(file_list)}<br>
           <b>Total size: {total_size:.1f} KB</b>
           """
       else:
           file_display.value = "<b>No files uploaded yet</b>"

   data_uploader.observe(update_file_display, names='value')

   # Training function
   def start_training(b):
       with output_area:
           try:
               # Update trainer parameters
               trainer.config['epochs'] = epochs_slider.value
               trainer.config['batch_size'] = batch_size_slider.value
               trainer.config['paraphrase_rounds'] = paraphrase_slider.value
               trainer.config['use_attention'] = attention_checkbox.value

               # Set the uploader
               trainer.data_uploader = data_uploader

               print("🚀 Starting training process...")
               trainer.load_data_from_ui()
               trainer.apply_data_augmentation()
               trainer.build_model()
               trainer.train_model()
               print("✅ Training completed successfully!")

           except Exception as e:
               print(f"❌ Error during training: {e}")
               import traceback
               traceback.print_exc()

   train_button.on_click(start_training)

   # Layout
   ui = widgets.VBox([
       widgets.HTML("<h3>🎯 Hybrid Political Text Classification Model</h3>"),
       widgets.HTML("<h4>📁 File Upload</h4>"),
       data_uploader,
       file_display,
       widgets.HTML("<h4>⚙️ Training Parameters</h4>"),
       epochs_slider,
       batch_size_slider,
       paraphrase_slider,
       attention_checkbox,
       model_suffix,
       train_button,
       output_area
   ])

   display(ui)
   return trainer

In [ ]:
def analyze_training_summary(trainer):
    """Comprehensive analysis with built-in column fixing AND correlations"""

    print("=" * 80)
    print("📊 HYBRID MODEL TRAINING SUMMARY REPORT")
    print("=" * 80)

    # 1. DATASET ANALYSIS
    print("\n📁 DATASET COMPOSITION")
    print("-" * 40)

    if hasattr(trainer, 'train_dataset') and trainer.train_dataset:
        train_df = trainer.train_dataset.dataframe.copy()
        test_df = trainer.test_dataset.dataframe.copy() if hasattr(trainer, 'test_dataset') else pd.DataFrame()

        # FIX BOOLEAN COLUMNS AUTOMATICALLY
        if 'is_prototype' in train_df.columns:
            train_df['is_prototype'] = train_df['is_prototype'].fillna(False).astype(bool)
        else:
            train_df['is_prototype'] = False

        if 'is_paraphrased' in train_df.columns:
            train_df['is_paraphrased'] = train_df['is_paraphrased'].fillna(False).astype(bool)
        else:
            train_df['is_paraphrased'] = False

        # Basic statistics
        print(f"📈 Training samples: {len(train_df):,}")
        print(f"📉 Test samples: {len(test_df):,}")
        print(f"📊 Total samples: {len(train_df) + len(test_df):,}")

        # Overall composition
        prototypes = train_df['is_prototype'].sum()
        non_prototypes = (~train_df['is_prototype']).sum()
        print(f"⚓ Prototype anchors: {prototypes}")
        print(f"📝 Real texts: {non_prototypes}")

        original_texts = (~train_df['is_paraphrased'] & ~train_df['is_prototype']).sum()
        paraphrased_texts = train_df['is_paraphrased'].sum()
        print(f"📄 Original texts: {original_texts}")
        print(f"🔄 Paraphrased texts: {paraphrased_texts}")
        if original_texts > 0:
            augmentation_ratio = paraphrased_texts / original_texts
            print(f"📈 Augmentation ratio: {augmentation_ratio:.1f}x")

        # Source breakdown
        if 'source' in train_df.columns:
            print(f"\n📂 FILES PROCESSED:")

            for source in train_df['source'].unique():
                if 'prototype' not in source.lower():
                    source_df = train_df[train_df['source'] == source]

                    original = ((source_df['source'] == source) &
                              (~source_df['is_paraphrased']) &
                              (~source_df['is_prototype'])).sum()
                    paraphrased = ((source_df['source'] == source) &
                                 (source_df['is_paraphrased'])).sum()
                    total = len(source_df)

                    print(f"  📄 {source}")
                    print(f"     • Original texts: {original}")
                    print(f"     • Paraphrased texts: {paraphrased}")
                    print(f"     • Total from source: {total}")

                    if original > 0:
                        ratio = paraphrased / original
                        print(f"     • Augmentation: {ratio:.1f}x")

        # Goal distribution analysis
        print(f"\n🎯 GOAL TYPE DISTRIBUTIONS:")
        non_prototype_df = train_df[~train_df['is_prototype']]

        if len(non_prototype_df) > 0:
            goal_stats = non_prototype_df[['prog_human', 'mant_human', 'prot_human']].describe()

            for goal in ['prog_human', 'mant_human', 'prot_human']:
                goal_name = goal.replace('_human', '').title()
                print(f"  📊 {goal_name}:")
                print(f"     • Mean: {goal_stats.loc['mean', goal]:.3f}")
                print(f"     • Std:  {goal_stats.loc['std', goal]:.3f}")
                print(f"     • Range: [{goal_stats.loc['min', goal]:.3f}, {goal_stats.loc['max', goal]:.3f}]")

    # 2. MODEL STATUS
    print(f"\n🏗️  MODEL STATUS")
    print("-" * 40)

    if hasattr(trainer, 'model') and trainer.model:
        model = trainer.model
        total_params = sum(p.numel() for p in model.parameters())
        trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

        print(f"🧠 Total parameters: {total_params:,}")
        print(f"🔧 Trainable parameters: {trainable_params:,}")
        print(f"📐 Model size: ~{total_params * 4 / 1024 / 1024:.1f} MB")
        print(f"✅ Model is loaded and ready")

        # Configuration
        print(f"\n🔧 CONFIGURATION:")
        for key, value in trainer.config.items():
            print(f"  • {key}: {value}")

    # 3. MODEL PERFORMANCE ON TEST SET (WITH CORRELATIONS)
    print(f"\n📈 MODEL PERFORMANCE")
    print("-" * 40)

    if hasattr(trainer, 'model') and hasattr(trainer, 'test_dataset') and trainer.model is not None and len(test_df) > 0:
        print("🔮 Generating predictions on test set...")

        # Get test predictions
        test_texts = test_df['item'].tolist()
        try:
            predictions = trainer.predict(test_texts)

            # Add predictions to test dataframe
            test_results = test_df.copy()
            test_results['prog_predicted'] = predictions[:, 0]
            test_results['mant_predicted'] = predictions[:, 1]
            test_results['prot_predicted'] = predictions[:, 2]

            # Calculate overall correlations
            overall_corr_prog = test_results['prog_human'].corr(test_results['prog_predicted'])
            overall_corr_mant = test_results['mant_human'].corr(test_results['mant_predicted'])
            overall_corr_prot = test_results['prot_human'].corr(test_results['prot_predicted'])

            print(f"\n📊 OVERALL TEST CORRELATIONS:")
            print(f"  🎯 Progressive: {overall_corr_prog:.3f}")
            print(f"  ⚖️  Maintenance: {overall_corr_mant:.3f}")
            print(f"  🛡️  Protective:  {overall_corr_prot:.3f}")
            print(f"  📈 Average:     {(overall_corr_prog + overall_corr_mant + overall_corr_prot)/3:.3f}")

            # Calculate correlations by source
            if 'source' in test_results.columns:
                print(f"\n📂 CORRELATIONS BY SOURCE:")
                print("-" * 35)

                correlation_summary = []

                for source in test_results['source'].unique():
                    source_df = test_results[test_results['source'] == source]

                    if len(source_df) >= 3:  # Need at least 3 samples for correlation
                        corr_prog = source_df['prog_human'].corr(source_df['prog_predicted'])
                        corr_mant = source_df['mant_human'].corr(source_df['mant_predicted'])
                        corr_prot = source_df['prot_human'].corr(source_df['prot_predicted'])
                        avg_corr = (corr_prog + corr_mant + corr_prot) / 3

                        # Handle NaN correlations
                        corr_prog = corr_prog if not pd.isna(corr_prog) else 0.0
                        corr_mant = corr_mant if not pd.isna(corr_mant) else 0.0
                        corr_prot = corr_prot if not pd.isna(corr_prot) else 0.0
                        avg_corr = avg_corr if not pd.isna(avg_corr) else 0.0

                        print(f"📄 {source[:40]}{'...' if len(source) > 40 else ''}")
                        print(f"   Samples: {len(source_df)}")
                        print(f"   Progressive: {corr_prog:.3f}")
                        print(f"   Maintenance: {corr_mant:.3f}")
                        print(f"   Protective:  {corr_prot:.3f}")
                        print(f"   Average:     {avg_corr:.3f}")
                        print()

                        correlation_summary.append({
                            'source': source,
                            'n_samples': len(source_df),
                            'prog_corr': corr_prog,
                            'mant_corr': corr_mant,
                            'prot_corr': corr_prot,
                            'avg_corr': avg_corr
                        })
                    else:
                        print(f"📄 {source}: Too few samples ({len(source_df)}) for correlation")

                # Summary of correlations
                if correlation_summary:
                    corr_df = pd.DataFrame(correlation_summary)
                    print(f"📊 CORRELATION SUMMARY:")
                    print(f"   Best overall performance: {corr_df.loc[corr_df['avg_corr'].idxmax(), 'source'][:30]}... ({corr_df['avg_corr'].max():.3f})")
                    print(f"   Worst overall performance: {corr_df.loc[corr_df['avg_corr'].idxmin(), 'source'][:30]}... ({corr_df['avg_corr'].min():.3f})")
                    print(f"   Progressive correlations: {corr_df['prog_corr'].mean():.3f} ± {corr_df['prog_corr'].std():.3f}")
                    print(f"   Maintenance correlations: {corr_df['mant_corr'].mean():.3f} ± {corr_df['mant_corr'].std():.3f}")
                    print(f"   Protective correlations:  {corr_df['prot_corr'].mean():.3f} ± {corr_df['prot_corr'].std():.3f}")

            # Calculate error statistics
            test_results['prog_error'] = abs(test_results['prog_predicted'] - test_results['prog_human'])
            test_results['mant_error'] = abs(test_results['mant_predicted'] - test_results['mant_human'])
            test_results['prot_error'] = abs(test_results['prot_predicted'] - test_results['prot_human'])

            print(f"\n📏 PREDICTION ERRORS (Mean Absolute Error):")
            print(f"   Progressive: {test_results['prog_error'].mean():.3f} ± {test_results['prog_error'].std():.3f}")
            print(f"   Maintenance: {test_results['mant_error'].mean():.3f} ± {test_results['mant_error'].std():.3f}")
            print(f"   Protective:  {test_results['prot_error'].mean():.3f} ± {test_results['prot_error'].std():.3f}")

            # Store results for potential download
            trainer.test_results = test_results

        except Exception as e:
            print(f"❌ Error generating predictions: {e}")
            print("   Model may still be training or not properly loaded")

    elif hasattr(trainer, 'model') and trainer.model is not None:
        print("📊 Model trained but no test data available for evaluation")
    else:
        print("📊 Model not yet trained - no performance metrics available")

    print("\n" + "=" * 80)
    return True

print("✅ analyze_training_summary function updated with correlations")

✅ analyze_training_summary function updated with correlations


In [ ]:
def create_training_monitor():
    """Create a widget to monitor training progress and show summary"""
    import ipywidgets as widgets
    from IPython.display import display

    # Summary button
    summary_button = widgets.Button(
        description='📊 Show Training Summary',
        button_style='info',
        icon='chart-bar'
    )

    # Analysis output
    analysis_output = widgets.Output()

    def show_summary(b):
        with analysis_output:
            from IPython.display import clear_output
            clear_output()
            try:
                # Use the existing analyze_training_summary function
                analyze_training_summary(trainer)
            except Exception as e:
                print(f"❌ Error generating summary: {e}")
                print("Make sure training has started and data is loaded.")
                import traceback
                traceback.print_exc()

    summary_button.on_click(show_summary)

    # Progress tracking
    progress_output = widgets.Output()

    # Real-time stats
    stats_html = widgets.HTML(value="<b>Training not started</b>")

    display(widgets.VBox([
        widgets.HTML("<h3>📊 Training Analysis Dashboard</h3>"),
        summary_button,
        stats_html,
        analysis_output,
        progress_output
    ]))

    return summary_button, analysis_output, progress_output, stats_html

def enhanced_training_tracker(trainer):
    """Enhanced training function with real-time monitoring"""

    # Store original train_model method
    original_train = trainer.train_model

    def monitored_train_model():
        """Wrapper that adds monitoring to training"""

        # Initialize tracking
        trainer.training_history = {
            'train_loss': [],
            'val_loss': [],
            'correlations': [],
            'epoch_times': []
        }

        print("🚀 Enhanced training with monitoring started...")

        # Call original training with monitoring
        result = original_train()

        # Generate final summary using existing function
        print("\n" + "🎉" * 20)
        print("TRAINING COMPLETED!")
        print("🎉" * 20)
        analyze_training_summary(trainer)

        return result

    # Replace the method
    trainer.train_model = monitored_train_model
    return trainer

print("✅ Missing functions added - uses your existing analyze_training_summary!")

✅ Missing functions added - uses your existing analyze_training_summary!


In [ ]:
# -*- coding: utf-8 -*-


# Quick start function
def quick_train_hybrid_model(data_files,
                            embedding_models=['paraphrase-multilingual-mpnet-base-v2'],
                            paraphrase_rounds=2,
                            epochs=50):
    """Quick training function for programmatic use"""
    trainer = HybridModelTrainer(
        embedding_models=embedding_models,
        paraphrase_rounds=paraphrase_rounds,
        epochs=epochs
    )

    # Load data programmatically
    datasets = []
    for filename in data_files:
        df = pd.read_csv(filename, dtype={'item': str})
        df = df[['item', 'prog_human', 'mant_human', 'prot_human']].copy()
        df['item'] = df['item'].astype(str)
        df[['prog_human', 'mant_human', 'prot_human']] = df[['prog_human', 'mant_human', 'prot_human']] / 7.0
        df['source'] = filename

        train = df.sample(frac=0.8, random_state=42)
        test = df.drop(train.index)
        datasets.append((train, test))

    train_df = pd.concat([d[0] for d in datasets], ignore_index=True)
    test_df = pd.concat([d[1] for d in datasets], ignore_index=True)

    trainer.train_dataset = HybridTextDataset(train_df, add_prototypes=True)
    trainer.test_dataset = HybridTextDataset(test_df, add_prototypes=False)

    # Train the model
    trainer.apply_data_augmentation()
    trainer.build_model()
    trainer.train_model()

    return trainer

if __name__ == "__main__":
    # For Jupyter/Colab usage
    print("Hybrid Political Text Classification Model")
    print("Use create_hybrid_model_interface() for interactive training")
    print("Use quick_train_hybrid_model(data_files) for programmatic training")

Hybrid Political Text Classification Model
Use create_hybrid_model_interface() for interactive training
Use quick_train_hybrid_model(data_files) for programmatic training


In [ ]:
trainer = create_flexible_hybrid_interface()
# trainer = create_flexible_hybrid_interface()
# trainer = create_hybrid_model_interface()

Initialized trainer with device: cuda
Configuration: {'embedding_models': ['paraphrase-multilingual-mpnet-base-v2'], 'hidden_layers': [512, 256, 128], 'use_attention': True, 'paraphrase_rounds': 0, 'epochs': 50, 'batch_size': 32, 'learning_rate': 0.001}


In [ ]:
# Cell 1: Create monitoring dashboard
monitor_button, analysis_area, progress_area, stats_display = create_training_monitor()

# Cell 2: Manual summary check (run anytime during/after training)
def quick_summary():
    """Quick training summary"""
    if 'trainer' in globals():
        analyze_training_summary(trainer)
    else:
        print("❌ Trainer not found. Make sure training interface is initialized.")

# Run this anytime to check progress
quick_summary()

📊 HYBRID MODEL TRAINING SUMMARY REPORT

📁 DATASET COMPOSITION
----------------------------------------
📈 Training samples: 383
📉 Test samples: 95
📊 Total samples: 478
⚓ Prototype anchors: 9
📝 Real texts: 374
📄 Original texts: 374
🔄 Paraphrased texts: 0
📈 Augmentation ratio: 0.0x

📂 FILES PROCESSED:
  📄 AllHumanGoalRatings2012_reliable.csv
     • Original texts: 50
     • Paraphrased texts: 0
     • Total from source: 50
     • Augmentation: 0.0x
  📄 AllHumanGoalRatingsBE_reliable.csv
     • Original texts: 38
     • Paraphrased texts: 0
     • Total from source: 38
     • Augmentation: 0.0x
  📄 AllHumanGoalRatingsGerman_reliable.csv
     • Original texts: 70
     • Paraphrased texts: 0
     • Total from source: 70
     • Augmentation: 0.0x
  📄 AllHumanGoalRatingsManifesto_reliable_clean.csv
     • Original texts: 86
     • Paraphrased texts: 0
     • Total from source: 86
     • Augmentation: 0.0x
  📄 AllHumanGoalRatingsNL_reliable.csv
     • Original texts: 64
     • Paraphrased texts

/tmp/ipython-input-18-3268128282.py:18: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_df['is_prototype'] = train_df['is_prototype'].fillna(False).astype(bool)



📊 OVERALL TEST CORRELATIONS:
  🎯 Progressive: 0.433
  ⚖️  Maintenance: 0.545
  🛡️  Protective:  0.567
  📈 Average:     0.515

📂 CORRELATIONS BY SOURCE:
-----------------------------------
📄 AllHumanGoalRatings2012_reliable.csv
   Samples: 13
   Progressive: 0.770
   Maintenance: 0.701
   Protective:  0.673
   Average:     0.715

📄 AllHumanGoalRatingsBE_reliable.csv
   Samples: 10
   Progressive: 0.644
   Maintenance: 0.481
   Protective:  0.545
   Average:     0.556

📄 AllHumanGoalRatingsGerman_reliable.csv
   Samples: 17
   Progressive: 0.305
   Maintenance: 0.687
   Protective:  0.812
   Average:     0.601

📄 AllHumanGoalRatingsManifesto_reliable_cl...
   Samples: 22
   Progressive: 0.022
   Maintenance: 0.333
   Protective:  0.406
   Average:     0.254

📄 AllHumanGoalRatingsNL_reliable.csv
   Samples: 16
   Progressive: 0.662
   Maintenance: 0.688
   Protective:  0.637
   Average:     0.662

📄 AllHumanGoalRatingsSpeaches_reliable.csv
   Samples: 17
   Progressive: 0.175
   Maintena

In [ ]:
def save_quick_summary(trainer, filename="training_summary"):
    """Save the output of quick_summary to a file"""
    import io
    import sys
    from contextlib import redirect_stdout
    from datetime import datetime

    # Create timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    # Capture the output
    summary_output = io.StringIO()

    try:
        with redirect_stdout(summary_output):
            if 'trainer' in globals():
                analyze_training_summary(trainer)
            else:
                print("❌ Trainer not found. Make sure training interface is initialized.")

        # Get the captured output
        summary_text = summary_output.getvalue()

        # Save to file
        output_filename = f"{filename}_{timestamp}.txt"
        with open(output_filename, 'w', encoding='utf-8') as f:
            f.write(f"Training Summary Report\n")
            f.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write("=" * 80 + "\n\n")
            f.write(summary_text)

        print(f"✅ Summary saved to: {output_filename}")

        # Download in Colab
        from google.colab import files
        files.download(output_filename)

        return output_filename

    except Exception as e:
        print(f"❌ Error saving summary: {e}")
        return None

# Usage:
save_quick_summary(trainer, "my_model_summary")

/tmp/ipython-input-18-3268128282.py:18: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_df['is_prototype'] = train_df['is_prototype'].fillna(False).astype(bool)


✅ Summary saved to: my_model_summary_20250708_133636.txt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

'my_model_summary_20250708_133636.txt'

In [ ]:
def save_model_for_reuse(trainer, model_name="political_classifier"):
    """Save model in multiple formats for easy reuse"""
    from datetime import datetime
    import torch
    import json

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    base_name = f"{model_name}_{timestamp}"

    try:
        if not hasattr(trainer, 'model') or trainer.model is None:
            print("❌ No trained model found!")
            return None

        print("💾 Saving model for reuse...")

        # 1. Save complete model (easiest to load)
        complete_path = f"{base_name}_complete.pt"
        torch.save(trainer.model, complete_path)

        # 2. Save state dict + config (more flexible)
        checkpoint_path = f"{base_name}_checkpoint.pth"
        checkpoint = {
            'model_state_dict': trainer.model.state_dict(),
            'model_config': {
                'embedding_models': trainer.config['embedding_models'],
                'hidden_layers': trainer.config['hidden_layers'],
                'use_attention': trainer.config['use_attention'],
                'dropout_rate': trainer.config.get('dropout_rate', 0.2),
                'trainable_embeddings': True,
                'batch_norm': trainer.config.get('batch_norm', True),
                'attention_heads': trainer.config.get('attention_heads', 1)
            },
            'training_config': trainer.config,
            'timestamp': timestamp
        }
        torch.save(checkpoint, checkpoint_path)

        # 3. Save usage instructions
        instructions_path = f"{base_name}_usage_guide.py"
        usage_code = f'''"""
Political Text Classification Model - Usage Guide
Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
Model: {model_name}
"""

import torch
import numpy as np

# ==================== SIMPLE USAGE (RECOMMENDED) ====================

def load_model():
    """Load the saved model"""
    model = torch.load('{complete_path}', map_location='cpu')
    model.eval()
    return model

def score_political_texts(texts):
    """Score political texts - MAIN FUNCTION TO USE"""
    model = load_model()

    if isinstance(texts, str):
        texts = [texts]

    with torch.no_grad():
        predictions = model(texts)
        predictions = predictions.cpu().numpy()

    results = []
    for i, text in enumerate(texts):
        prog, mant, prot = predictions[i]
        dominant_idx = predictions[i].argmax()
        dominant_goal = ['Progressive', 'Maintenance', 'Protective'][dominant_idx]
        confidence = predictions[i].max()

        results.append({{
            'text': text,
            'progressive': float(prog),
            'maintenance': float(mant),
            'protective': float(prot),
            'dominant_goal': dominant_goal,
            'confidence': float(confidence)
        }})

    return results

# ==================== USAGE EXAMPLES ====================

def example_usage():
    """Examples of how to use the model"""

    # Example 1: Score single text
    single_text = "We need comprehensive healthcare reform"
    result = score_political_texts(single_text)[0]
    print(f"Text: {{result['text']}}")
    print(f"Dominant goal: {{result['dominant_goal']}} (confidence: {{result['confidence']:.3f}})")
    print(f"All scores: Prog={{result['progressive']:.3f}}, Maint={{result['maintenance']:.3f}}, Prot={{result['protective']:.3f}}")

    # Example 2: Score multiple texts
    multiple_texts = [
        "We must reform our education system",
        "Traditional values are the foundation of society",
        "We need strong border security measures"
    ]

    results = score_political_texts(multiple_texts)
    for result in results:
        print(f"{{result['text']}} → {{result['dominant_goal']}}")

def score_csv_file(input_file, output_file, text_column='text'):
    """Score texts in a CSV file"""
    import pandas as pd

    # Read CSV
    df = pd.read_csv(input_file)

    if text_column not in df.columns:
        print(f"Error: Column '{{text_column}}' not found in CSV")
        available_columns = list(df.columns)
        print(f"Available columns: {{available_columns}}")
        return

    # Get texts and score them
    texts = df[text_column].tolist()
    results = score_political_texts(texts)

    # Add results to dataframe
    for i, result in enumerate(results):
        df.loc[i, 'progressive'] = result['progressive']
        df.loc[i, 'maintenance'] = result['maintenance']
        df.loc[i, 'protective'] = result['protective']
        df.loc[i, 'dominant_goal'] = result['dominant_goal']
        df.loc[i, 'confidence'] = result['confidence']

    # Save results
    df.to_csv(output_file, index=False)
    print(f"✅ Results saved to {{output_file}}")
    print(f"📊 Scored {{len(results)}} texts")

# ==================== QUICK START ====================

if __name__ == "__main__":
    print("Testing Political Text Classifier...")
    print("=" * 50)

    # Test texts
    test_texts = [
        "We need progressive tax reform",
        "Traditional marriage should be protected",
        "Strong military defense is essential"
    ]

    results = score_political_texts(test_texts)

    for result in results:
        print(f"Text: {{result['text']}}")
        print(f"→ {{result['dominant_goal']}} ({{result['confidence']:.3f}})")
        print(f"  Progressive: {{result['progressive']:.3f}}")
        print(f"  Maintenance: {{result['maintenance']:.3f}}")
        print(f"  Protective:  {{result['protective']:.3f}}")
        print("-" * 50)

# ==================== MODEL INFO ====================
# Model trained on: {datetime.now().strftime('%Y-%m-%d')}
# Training configuration: {json.dumps(trainer.config, indent=2)}
# Model parameters: {sum(p.numel() for p in trainer.model.parameters()):,}
'''

        with open(instructions_path, 'w', encoding='utf-8') as f:
            f.write(usage_code)

        # 4. Download files
        from google.colab import files
        files_to_download = [complete_path, checkpoint_path, instructions_path]

        print(f"✅ Model saved successfully!")
        print(f"📁 Files created:")
        for file_path in files_to_download:
            print(f"  • {file_path}")
            files.download(file_path)

        print(f"\n🚀 TO USE YOUR MODEL LATER:")
        print(f"1. Download and keep the files (especially the .pt file)")
        print(f"2. In any Python environment, run the usage guide code")
        print(f"3. Use: score_political_texts(['your text here'])")

        return {
            'complete_model': complete_path,
            'checkpoint': checkpoint_path,
            'usage_guide': instructions_path
        }

    except Exception as e:
        print(f"❌ Error saving model: {e}")
        import traceback
        traceback.print_exc()
        return None

# Quick save alternative (simpler)
def quick_save_model(trainer, model_name="political_model"):
    """Quick save of just the model"""
    import torch
    from datetime import datetime

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{model_name}_{timestamp}.pt"

    if not hasattr(trainer, 'model') or trainer.model is None:
        print("❌ No trained model found!")
        return None

    torch.save(trainer.model, filename)

    from google.colab import files
    files.download(filename)

    print(f"✅ Model saved as: {filename}")
    print(f"💡 To use later: model = torch.load('{filename}', map_location='cpu')")
    return filename

print("✅ Model saving functions defined!")
print("🚀 Now you can run:")
print("   save_model_for_reuse(trainer, 'my_classifier')")
print("   OR")
print("   quick_save_model(trainer, 'my_classifier')")

✅ Model saving functions defined!
🚀 Now you can run:
   save_model_for_reuse(trainer, 'my_classifier')
   OR
   quick_save_model(trainer, 'my_classifier')


In [ ]:
saved_files = save_model_for_reuse(trainer, "my_political_classifier")


💾 Saving model for reuse...
✅ Model saved successfully!
📁 Files created:
  • my_political_classifier_20250708_134836_complete.pt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

  • my_political_classifier_20250708_134836_checkpoint.pth


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

  • my_political_classifier_20250708_134836_usage_guide.py


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


🚀 TO USE YOUR MODEL LATER:
1. Download and keep the files (especially the .pt file)
2. In any Python environment, run the usage guide code
3. Use: score_political_texts(['your text here'])


In [ ]:
def save_training_data(trainer, data_name="training_data"):
    """Save the training and test datasets"""
    from datetime import datetime

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    files_created = []

    try:
        # Save training dataset
        if hasattr(trainer, 'train_dataset') and trainer.train_dataset:
            train_file = f"{data_name}_train_{timestamp}.csv"
            trainer.train_dataset.dataframe.to_csv(train_file, index=False, encoding='utf-8-sig')
            files_created.append(train_file)
            print(f"✅ Training data saved: {train_file}")

        # Save test dataset
        if hasattr(trainer, 'test_dataset') and trainer.test_dataset:
            test_file = f"{data_name}_test_{timestamp}.csv"
            trainer.test_dataset.dataframe.to_csv(test_file, index=False, encoding='utf-8-sig')
            files_created.append(test_file)
            print(f"✅ Test data saved: {test_file}")

        # Download files
        from google.colab import files
        for file_path in files_created:
            files.download(file_path)

        return files_created

    except Exception as e:
        print(f"❌ Error saving data: {e}")
        return []

# Run this if you want to save the training data too:
save_training_data(trainer, "my_political_data")

✅ Training data saved: my_political_data_train_20250708_140022.csv
✅ Test data saved: my_political_data_test_20250708_140022.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

['my_political_data_train_20250708_140022.csv',
 'my_political_data_test_20250708_140022.csv']